In [1]:
import pandas as pd
import os

def preprocess_data():
    """
    Memuat data log, membersihkan, melakukan pra-pemrosesan, 
    dan menyimpan hasilnya.
    """
    # Tentukan path relatif terhadap lokasi script
    input_path = os.path.join('..', 'output', 'action_shelf_log.csv')
    output_dir = 'preprocessed_data'
    output_path = os.path.join(output_dir, 'cleaned_action_shelf_log.csv')

    # Buat direktori output jika belum ada
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Membaca data dari: {input_path}")
    try:
        df = pd.read_csv(input_path)
    except FileNotFoundError:
        print(f"Error: File tidak ditemukan di {input_path}")
        print("Pastikan file 'action_shelf_log.csv' ada di dalam folder 'full_pipeline/output/'.")
        return

    # 1. Urutkan data berdasarkan pelanggan (pid) dan waktu (frame)
    df_sorted = df.sort_values(by=['pid', 'frame']).reset_index(drop=True)
    print(f"Data diurutkan berdasarkan 'pid' dan 'frame'. Jumlah baris awal: {len(df_sorted)}")

    # 2. Tangani duplikasi frame dengan prioritas aksi
    # Aksi dengan intensi lebih tinggi diberi prioritas lebih tinggi.
    action_priority = [
        'Retract From Shelf',
        'Reach To Shelf',
        'Inspect Product',
        'Inspect Shelf',
        'Hand In Shelf',
        'Background'
    ]
    
    # Ubah kolom 'action' menjadi tipe kategori dengan urutan prioritas
    df_sorted['action'] = pd.Categorical(df_sorted['action'], categories=action_priority, ordered=True)

    # Urutkan lagi dengan prioritas aksi, lalu hapus duplikat
    df_cleaned = df_sorted.sort_values(by=['pid', 'frame', 'action'])
    df_cleaned = df_cleaned.drop_duplicates(subset=['pid', 'frame'], keep='first')
    
    print(f"Duplikasi frame ditangani. Jumlah baris setelah pembersihan: {len(df_cleaned)}")

    # 3. Simpan data yang sudah bersih
    df_cleaned.to_csv(output_path, index=False)
    print(f"Data bersih telah disimpan di: {output_path}")

In [2]:
if __name__ == '__main__':
    preprocess_data()

Membaca data dari: ..\output\action_shelf_log.csv
Data diurutkan berdasarkan 'pid' dan 'frame'. Jumlah baris awal: 1893
Duplikasi frame ditangani. Jumlah baris setelah pembersihan: 1415
Data bersih telah disimpan di: preprocessed_data\cleaned_action_shelf_log.csv
